In [51]:
print("Hello World")

Hello World

In [52]:
include("parser.jl")

r, c, b, m, t = readfile("Instances/gap1.txt",2);

In [53]:
variables = ["r", "c", "b", "m", "t"]
values = [r, c, b, m, t]

for i in 1:length(variables)
    println("$(variables[i]) : $(values[i])")
end

r : Any[23 10 15 13 17 10 13 6 9 21 20 7 9 25 8; 17 13 8 23 11 18 7 22 13 5 24 24 15 10 22; 22 17 22 23 20 11 17 25 23 9 22 20 15 9 25; 5 19 25 16 15 10 18 9 11 20 7 21 15 8 25; 22 9 10 23 19 21 17 15 15 17 25 19 10 9 21]
c : Any[22 21 16 17 21 15 17 22 22 25 18 20 24 15 22; 23 24 19 15 16 21 15 25 16 21 20 19 16 23 20; 21 20 21 25 21 20 21 19 17 16 25 19 15 15 15; 17 21 25 25 23 22 20 19 20 25 15 20 21 25 23; 15 25 23 19 17 17 25 24 24 17 24 19 18 19 16]
b : Any[32, 37, 44, 35, 40]
m : 5
t : 15


In [54]:
# r : quantité de ressources consommées par tâche (matrix)
r

5×15 Matrix{Any}:
 23  10  15  13  17  10  13   6   9  21  20   7   9  25   8
 17  13   8  23  11  18   7  22  13   5  24  24  15  10  22
 22  17  22  23  20  11  17  25  23   9  22  20  15   9  25
  5  19  25  16  15  10  18   9  11  20   7  21  15   8  25
 22   9  10  23  19  21  17  15  15  17  25  19  10   9  21

In [55]:
# c : gain par tâche pour chaque agent (matrix)
c

5×15 Matrix{Any}:
 22  21  16  17  21  15  17  22  22  25  18  20  24  15  22
 23  24  19  15  16  21  15  25  16  21  20  19  16  23  20
 21  20  21  25  21  20  21  19  17  16  25  19  15  15  15
 17  21  25  25  23  22  20  19  20  25  15  20  21  25  23
 15  25  23  19  17  17  25  24  24  17  24  19  18  19  16

In [56]:
# b : quantité de ressource de l'agent i (vect)
b

5-element Vector{Any}:
 32
 37
 44
 35
 40

In [57]:
# m : nombre d'agents (int) --> nombre de lignes
m

5

In [58]:
# t : nombre de tâches (int) --> nombre de colonnes
t

15

### Heuristique gloutonne

In [121]:
function greedy(file, instance)

    r, c, b, m, t = readfile(file, instance);

    current_load = fill(0, m)
    assignment = fill(0, t)
    cost = 0

    end_condition = fill(100, m, t)

    while r != end_condition
        # Booléen pour sortir de la boucle si rien ne se passe
        feasibility = false
        for i in 1:m
            # Si la tâche qui nécessite le moins de ressources satisfait la contrainte
            min = minimum(r[i, :])
            amin = argmin(r[i, :])
            if current_load[i] + min <= b[i]
                feasibility = true
                # On assigne la nouvelle tâche à l'agent
                assignment[amin] = i
                # On augmente sa charge
                current_load[i] += min
                # On ajoute le coût associée
                cost += c[i, amin]
                # On enlève la tâche des tâches à assigner
                r[:, amin] .= 100
            end
        end
        if feasibility == false
            # Pas de solution, on sort de la boucle
            # print("Pas de solution.")
            # break
            return 0
        end

    end
    # println(assignment)
    # println(cost)
    return cost
end

greedy (generic function with 1 method)

### Create CSV

In [133]:
using DataFrames
using CSV

In [83]:
files = readdir("Instances")

16-element Vector{String}:
 "gap1.txt"
 "gap10.txt"
 "gap11.txt"
 "gap12.txt"
 "gap2.txt"
 "gap3.txt"
 "gap4.txt"
 "gap5.txt"
 "gap6.txt"
 "gap7.txt"
 "gap8.txt"
 "gap9.txt"
 "gapa.txt"
 "gapb.txt"
 "gapc.txt"
 "gapd.txt"

In [125]:
greedy("Instances/gap1.txt", 0)

294

In [113]:
df = DataFrame(id = Int[], file = String[], instance = Int[], performance = Int[])

Row,id,file,instance,performance
,Int64,String,Int64,Int64


In [116]:
push!(df, (1, "test", 1, 10))

Row,id,file,instance,performance
,Int64,String,Int64,Int64
1,1,test,1,10
2,1,test,1,10
3,1,test,1,10


In [138]:
df = DataFrame(id = Int[], file = String[], instance = Int[], greedy_worker = Int[])
id = 0

for file in files
    path = "Instances/"*file
    nb_instances = parse(Int, readline(path))
    for instance in 0:nb_instances-1
        greedy_worker = greedy(path, instance)
        push!(df, (id, path, instance, greedy_worker))
        id += 1
    end
end

In [139]:
CSV.write("output_csv", df)

"output_csv"

In [ ]:
using CSV
using DataFrames

function read_instances_from_txt(filename)
    # Ouvre le fichier et lit chaque ligne (suppose chaque instance est une ligne)
    instances = []
    open(filename, "r") do file
        for line in eachline(file)
            push!(instances, chomp(line))  # Enlève les retours à la ligne
        end
    end
    return instances
end

function generate_csv_from_txt_files(folder_path, output_csv)
    # Crée un DataFrame pour stocker toutes les instances
    df = DataFrame(id = Int[], file = String[], instance = String[])
    
    # Parcourt chaque fichier txt dans le dossier
    txt_files = filter(x -> endswith(x, ".txt"), readdir(folder_path, join=true))
    
    id_counter = 1  # Initialisation d'un compteur d'ID pour chaque instance
    for file in txt_files
        instances = read_instances_from_txt(file)
        for instance in instances
            push!(df, (id_counter, file, instance))
            id_counter += 1
        end
    end
    
    # Sauvegarde toutes les données dans un fichier CSV
    CSV.write(output_csv, df)
end

# Appel de la fonction avec le chemin de votre dossier et le nom de sortie du CSV
generate_csv_from_txt_files("chemin/vers/dossier/txt_files", "instances.csv")

### Voisinages

### Recuit

In [ ]:
function recuit(solution_initiale)

end

In [71]:
greedy("Instances/gap2.txt", 2)

[3, 1, 1, 2, 5, 2, 3, 2, 5, 3, 2, 4, 4, 4, 1, 3, 5, 4, 1, 5]
358


In [61]:
cost

266

### Heuristique gloutonne

In [62]:
ratio = c./r

5×15 Matrix{Float64}:
 0.956522  2.1      1.06667   1.30769   …  2.66667  0.6      2.75
 1.35294   1.84615  2.375     0.652174     1.06667  2.3      0.909091
 0.954545  1.17647  0.954545  1.08696      1.0      1.66667  0.6
 3.4       1.10526  1.0       1.5625       1.4      3.125    0.92
 0.681818  2.77778  2.3       0.826087     1.8      2.11111  0.761905

In [63]:
argmax(ratio)

CartesianIndex(2, 10)

In [64]:
current_ressource = zeros(m)

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [65]:
current_ressource[1] < b[1]

true

In [66]:
ratio[10,:]

BoundsError: BoundsError: attempt to access 5×15 Matrix{Float64} at index [10, 1:15]

In [67]:
ratio

5×15 Matrix{Float64}:
 0.956522  2.1      1.06667   1.30769   …  2.66667  0.6      2.75
 1.35294   1.84615  2.375     0.652174     1.06667  2.3      0.909091
 0.954545  1.17647  0.954545  1.08696      1.0      1.66667  0.6
 3.4       1.10526  1.0       1.5625       1.4      3.125    0.92
 0.681818  2.77778  2.3       0.826087     1.8      2.11111  0.761905